This notebook contains the code to write for each step of the project. 

In [7]:
#Visualizing the video
import cv2
video_path = 'data\data_v\parking_crop_loop.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video source.")
    exit()

ret=True
while ret:
    ret, frame= cap.read()
    
    if not ret:
        print("Error: Could not read frame.")
        break
    
    cv2.imshow('Video Frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

: 

In [ ]:
#load the mask 
import cv2
mask = 'mask\mask_crop.png'
mask = cv2.imread(mask, 0)

In [ ]:
#detect the parking spots and create the bounding boxes using the mask
from utils import get_parking_spots_boxes
connected_componets = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)
parking_spots = get_parking_spots_boxes(connected_componets)
print(parking_spots[0]) #getting the value of the coordonnées of the first parking spot

[70, 0, 131, 56]


In [ ]:
#adding to the video the bounding boxes in Blue
for spot_idx, spot in enumerate(parking_spots):
        x1,y1, w, h = spot
        frame = cv2.rectangle(frame, (x1,y1), (x1+w,y1+h), (255,0,0))

In [ ]:
#classifying each blue bounding box to know if it is available or not 
#Red => Not available, Green => Available
from utils import empty_or_not
spot_crop = frame[y1:y1+h, x1:x1+w]
spot_status = empty_or_not(spot_crop)
if spot_status:
    frame = cv2.rectangle(frame, (x1,y1), (x1+w,y1+h), (0,255,0),2)
else:
    frame = cv2.rectangle(frame, (x1,y1), (x1+w,y1+h), (0,0,255),2)

In [ ]:
#Optimizing to make it quicker => using logic of when parking a car
#=> Classifying every 30 frames

spots_status = [None for j in parking_spots]
step = 30
spots_status[spot_idx] = spot_status
#adding an if statement to update the spot_status every 30 frames
#if frame_number % step == 0:

In [ ]:
#to improve more the performance: considering the work on only the changing parking spots not all of them
#defining the calc_diff function
import numpy as np

def calc_diff(img1, img2):
    return np.abs(np.mean(img1) - np.mean(img2))

frame_number = 0
diffs = [None for j in parking_spots]
previous_frame = None
if frame_number % step == 0:
        previous_frame = frame.copy()

#adding this part 
if frame_number % step == 0: 
        for spot_idx, spot in enumerate(parking_spots):
            x1,y1, w, h = spot
            spot_crop = frame[y1:y1+h, x1:x1+w]
            diffs[spot_idx] = calc_diff(spot_crop, previous_frame[y1:y1+h, x1:x1+w])

In [ ]:
#By adding this code, it is now giving the best performance and a real time detetction
if previous_frame is None:
    arr = range(len(parking_spots))
else:
    arr = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4]
#and modifying the loop

In [ ]:
#adding the counter on the video
cv2.rectangle(frame, (80,20), (550,80), (0,0,0), -1)
cv2.putText(frame, 'Available Spots:{} / {}'.format(str(sum(spots_status)), str(len(parking_spots))), (100,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)